In [1]:
import pandas as pd
import numpy as np
import random
import os

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA

from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings("ignore")

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

# e 표시 안되게 
pd.options.display.float_format = '{:.2f}'.format

In [3]:
# 한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/H2GTRM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

## Data Load

from google.colab import drive
drive.mount('/content/drive')

#### colab
train = pd.read_csv('/content/drive/MyDrive/work/input/train.csv')
test = pd.read_csv('/content/drive/MyDrive/work/input/test.csv')
building_info = pd.read_csv('/content/drive/MyDrive/work/input/building_info.csv')
submission = pd.read_csv('/content/drive/MyDrive/work/input/sample_submission.csv')
train.shape, test.shape, building_info.shape, submission.shape

In [4]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
building_info = pd.read_csv('input/building_info.csv')

pred = pd.read_csv('output/20230828-3.csv')

train.shape, test.shape, building_info.shape, pred.shape

((204000, 10), (16800, 7), (100, 7), (16800, 2))

In [5]:
# 예측 연결
test = test.merge(pred, on=['num_date_time']).rename(columns={'answer':'전력소비량(kWh)'})

In [6]:
train[:1]

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.60,NaN,0.90,42.00,NaN,NaN,1085.28


In [7]:
building_info[:1]

,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,110634.00,39570.00,-,-,-


## EDA

In [ ]:
# 빌딩 데이터 정리
## null 처리
building_info['태양광용량(kW)'] = building_info['태양광용량(kW)'].str.replace('-','0').astype(float)
building_info['ESS저장용량(kWh)'] = building_info['ESS저장용량(kWh)'].str.replace('-','0').astype(float)
building_info['PCS용량(kW)'] = building_info['PCS용량(kW)'].str.replace('-','0').astype(float)

In [ ]:
# 시각화 스케일 조정 * 0.01
train['전력소비량(kWh)_plot'] = train['전력소비량(kWh)']*0.01
test['전력소비량(kWh)_plot'] = test['전력소비량(kWh)']*0.01

In [ ]:
#시계열 특성을 학습에 반영하기 위해 일시를 월, 일, 시간으로 나눕니다
train['month'] = train['일시'].apply(lambda x : int(x[4:6]))
train['day'] = train['일시'].apply(lambda x : int(x[6:8]))
train['time'] = train['일시'].apply(lambda x : int(x[9:11]))

test['month'] = test['일시'].apply(lambda x : int(x[4:6]))
test['day'] = test['일시'].apply(lambda x : int(x[6:8]))
test['time'] = test['일시'].apply(lambda x : int(x[9:11]))

In [ ]:
# 요일
train['weekday'] =  pd.to_datetime(train['일시'].str[:8]).dt.weekday
test['weekday'] =  pd.to_datetime(test['일시'].str[:8]).dt.weekday

In [ ]:
train['일시'] = pd.to_datetime(train['일시'])
test['일시'] = pd.to_datetime(test['일시'])

In [ ]:
# 정리된 빌딩과 merge
train = train.merge(building_info, on='건물번호')
test = test.merge(building_info, on='건물번호')

In [ ]:
pd.set_option('max_columns', 100)
# train[train['건물번호'] == 58].sort_values('전력소비량(kWh)', ascending=False)[:10]#.num_date_time.values
# train[train['건물번호'] == 73].sort_values('전력소비량(kWh)')[:5]

In [ ]:
train['date'] = pd.to_datetime(train['일시']).dt.strftime('%Y%m%d')
test['date'] = pd.to_datetime(test['일시']).dt.strftime('%Y%m%d')

In [ ]:
for i in range(13,15):
    
    if i not in [0]: 
        
        temp = train[train['건물번호'] == i][-21*24:-14*24]
        temp1 = temp[temp.weekday.isin([0,1,2,3,4])]
        temp2 = temp[temp.weekday.isin([5,6])]

        plt.figure(figsize=(20, 3))
        g = sns.lineplot(x=temp1['time'], y=temp1['전력소비량(kWh)_plot'], hue=temp1['date'])
#         sns.boxplot(x=temp["전력소비량(kWh)_plot"], saturation=0.99)
        plt.title(str(i)+'_'+temp[:1]['건물유형'].values[0])
#         plt.legend([],[], frameon=False)
        g.set_xticks(range(0,24))
        
        plt.figure(figsize=(20, 3))
        g = sns.lineplot(x=temp2['time'], y=temp2['전력소비량(kWh)_plot'], hue=temp2['date'])
        plt.title(str(i)+'_'+temp2[:1]['건물유형'].values[0]+'_휴일')
        g.set_xticks(range(0,24))
    
        plt.show()

In [ ]:
train['gubun'] = 'train'
test['gubun'] = 'test'
temp = pd.concat([train, test])

In [ ]:
for i in range(3, 4):
    plt.figure(figsize=(20, 3))    
    sns.lineplot(temp[temp['건물번호'] == i]['num_date_time'], temp[temp['건물번호'] == i]['전력소비량(kWh)_plot'], hue=temp[temp['건물번호'] == i]['gubun'])
    plt.title(i)
    plt.xticks(rotation=60)
    plt.show()

In [ ]:
for i in range(3, 4):
    plt.figure(figsize=(20, 3))  
#     temp.drop( temp[(temp['건물번호'].isin([3])) & (temp['month'].isin([6,8])) ].index, inplace=True)
    sns.lineplot(temp[temp['건물번호'] == i]['num_date_time'], temp[temp['건물번호'] == i]['전력소비량(kWh)_plot'], hue=temp[temp['건물번호'] == i]['gubun'])
    plt.title(i)
    plt.xticks(rotation=60)
    plt.show()

In [ ]:
plt.figure(figsize=(20, 3))  
temp1 = temp[(temp['건물번호'] == 58) & (temp['month'] == 7)][0*24:7*24]
temp1.drop( temp1[(train['day'].isin([5]))].index, inplace=True)
sns.lineplot(temp1['num_date_time'], temp1['전력소비량(kWh)_plot'], hue=temp1['gubun'])
plt.title(i)
plt.xticks(rotation=60)
plt.show()

In [ ]:
for i in range(93,101):
    plt.figure(figsize=(20, 3))    
    sns.lineplot(temp[temp['건물번호'] == i]['num_date_time'], temp[temp['건물번호'] == i]['전력소비량(kWh)_plot'], hue=temp[temp['건물번호'] == i]['gubun'])
    plt.title(i)
    plt.show()

In [ ]:
train[train['건물번호'] == 3][['date', 'time', '전력소비량(kWh)_plot']].pivot('date', 'time', '전력소비량(kWh)_plot')[-14:]

In [ ]:
temp = train[train['건물번호'] == 3][-7*24:]

In [ ]:
temp['전력소비량(kWh)'].std()

In [ ]:
train['date'] = pd.to_datetime(train['일시']).dt.strftime('%Y%m%d')

std = train.groupby(['건물번호']).agg({'전력소비량(kWh)':['std']}).reset_index()
std.columns = ['건물번호', 'std']

train_std = train.groupby(['건물번호','date','weekday']).agg({'전력소비량(kWh)':['std']}).reset_index()
train_std.columns = ['건물번호','date','weekday', 'date_std']

train_std = train_std.merge(std, on=['건물번호'])

train_std['std'] = np.where(train_std['date_std'] > train_std['std'], 0, 1)

In [ ]:
train_std[train_std['건물번호'] == 3][-21:-14]

In [ ]:
train_std[train_std['건물번호'] == 3][-14:-7]

In [ ]:
train_std[train_std['건물번호'] == 3][-7:]

In [ ]:
train['date'] = pd.to_datetime(train['일시']).dt.strftime('%Y%m%d')

std = train.groupby(['건물번호']).agg({'전력소비량(kWh)':['std']}).reset_index()
std.columns = ['건물번호', 'std']

train_std = train.groupby(['건물번호','date','weekday']).agg({'전력소비량(kWh)':['std']}).reset_index()
train_std.columns = ['건물번호','date','weekday', 'date_std']

train_std = train_std.merge(std, on=['건물번호'])

train_std['std'] = np.where(train_std['date_std'] > train_std['std'], 0, 1)

train_std_onthot = train_std.groupby(['건물번호','weekday']).mean()[['std']].reset_index()
train_std_onthot['std1'] = np.where(train_std_onthot['std'] > 0.5, 1, 0)

train_std_onthot[train_std_onthot['건물번호'] == 3]
train = train.merge(train_std_onthot[['건물번호','weekday', 'std1']], on=['건물번호','weekday'])

train.drop('date', axis=1, inplace=True)

In [ ]:
train_std_onthot[train_std_onthot['건물번호'] == 3]

In [ ]:
train_std[train_std['건물번호'] == 3][-7:]

In [ ]:
train

In [ ]:
18 목
19 금 -
22 월 -
23 화
24 수

In [ ]:
building_info[92:].get_du

In [ ]:
temp = train.groupby(['건물번호','건물유형', 'month']).agg({'전력소비량(kWh)':'mean'}).reset_index().pivot(['건물번호','건물유형'], 'month', '전력소비량(kWh)')
temp[87] = temp[8] - temp[7]
temp[86] = temp[8] - temp[6]
temp.sort_values(86, ascending=False).to_excel('월제외.xlsx')

In [ ]:
temp = train[train['건물번호'] == 1].groupby(['건물번호','건물유형', 'month']).agg({'기온(C)':'mean'}).reset_index().pivot(['건물번호','건물유형'], 'month', '기온(C)')
temp

In [ ]:
train[(train['건물번호'] == 1)][-7*24*2:-7*24]['기온(C)'].mean()

In [ ]:
train[(train['건물번호'] == 1)][-7*24:]['기온(C)'].mean()

In [ ]:
test[test['건물번호'] == 1]['기온(C)'].mean()

In [ ]:
train[['month', ]]

In [ ]:
for t in train['건물유형'].unique():
    print(t)
    temp = train[train['건물유형'] == t][['기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)','연면적(m2)', '냉방면적(m2)', '태양광용량(kW)',
       'ESS저장용량(kWh)', 'PCS용량(kW)','전력소비량(kWh)']]
    
    plt.figure(figsize=(9, 8))
#     sns.heatmap(temp.corr(), cmap='RdBu', annot=True,fmt=".1f")
    sns.pairplot(temp)
    plt.show()


In [ ]:
train.columns

In [ ]:
building_no = [1,16,24,32,37,45,53,61,69,77,85,93]

In [ ]:
train.columns

In [ ]:
# 호텔 # 39~100
# 95 > 98 유형이 낫다 

In [ ]:
train['습도(%)_plot'] = train['습도(%)'] * 0.2
test['습도(%)_plot'] = test['습도(%)'] * 0.2

In [ ]:
train.columns

In [ ]:
train.groupby(['건물유형', '건물번호','연면적(m2)', '냉방면적(m2)', '태양광용량(kW)',
       'ESS저장용량(kWh)', 'PCS용량(kW)']).agg({'전력소비량(kWh)_plot':['min','mean','max']}).reset_index().to_excel('건물별.xlsx')

In [ ]:
train[train['건물유형'] == '대학교'].pivot(index=['건물번호','month', 'day', 'weekday'], columns=['time'], values='전력소비량(kWh)_plot').reset_index().to_excel('대학교.xlsx')

In [ ]:
pd.set_option('max_columns', 30)
train.sort_values(['전력소비량(kWh)_plot'])[:10]

In [ ]:
train.pivot(index=['건물유형','건물번호','month', 'day', 'weekday'], columns=['time'], values='전력소비량(kWh)_plot')#.reset_index().to_excel('호텔및리조트.xlsx')

In [ ]:
train[train['건물유형'] == '호텔및리조트'].pivot(index=['건물번호','month', 'day', 'weekday'], columns=['time'], values='전력소비량(kWh)_plot').reset_index().to_excel('호텔및리조트.xlsx')

In [ ]:
bno = 95
for i in range(1, 7):  
    plt.figure(figsize=(16,3))
    temp = train[(train['건물번호'] == bno) & (train.weekday==0)] [-24*(i+1): -24*i].drop('num_date_time', axis=1) # & (train['month'] == 8)]
    display(temp[:1])
    plt.plot(temp['일시'], temp['풍속(m/s)'],linestyle='-', marker='o', label="풍속")
    plt.plot(temp['일시'], temp['강수량(mm)'],linestyle='-', marker='o', label="강수량")
    plt.plot(temp['일시'], temp['습도(%)_plot'],linestyle='-', marker='o', label="습도")
    plt.plot(temp['일시'], temp['기온(C)'],linestyle='-', marker='o', label="기온")
    
    plt.plot(temp['일시'], temp['전력소비량(kWh)_plot'],linestyle='-', label="전력소비량")
    
    plt.legend()
    plt.title(temp.weekday.unique())
    plt.show()

In [ ]:
bno = 2
for i in range(1, 2):  
    plt.figure(figsize=(16,4))
    temp = train[(train['건물번호'] == bno)] [-24*(i+1): -24*i] # & (train['month'] == 8)]
    display(temp[:1])
    plt.plot(temp['일시'], temp['풍속(m/s)'],linestyle='-', marker='o', label="풍속")
    plt.plot(temp['일시'], temp['강수량(mm)'],linestyle='-', marker='o', label="강수량")
    plt.plot(temp['일시'], temp['습도(%)_plot'],linestyle='-', marker='o', label="습도")
    plt.plot(temp['일시'], temp['기온(C)'],linestyle='-', marker='o', label="기온")
    
    plt.plot(temp['일시'], temp['전력소비량(kWh)_plot'],linestyle='-', label="전력소비량")
    
    plt.legend()
    plt.title(temp.weekday.unique())
    plt.show()

In [ ]:
temp = train[(train['건물번호'] == bno) &(train['month'] == 7)&(train['day'] == 6)]
display(temp['전력소비량(kWh)_plot'].values)
plt.figure(figsize=(16,4))
# plt.plot(temp['일시'], temp['풍속(m/s)'],linestyle='-', marker='o', label="풍속")
# plt.plot(temp['일시'], temp['강수량(mm)'],linestyle='-', marker='o', label="강수량")
# plt.plot(temp['일시'], temp['습도(%)_plot'],linestyle='-', marker='o', label="습도")
# plt.plot(temp['일시'], temp['기온(C)'],linestyle='-', marker='o', label="기온")

plt.plot(temp['일시'], temp['전력소비량(kWh)_plot'],linestyle='-', label="전력소비량")

plt.legend()
plt.title(temp.weekday.unique())
plt.show()

In [ ]:
train['week'] = train['일시'].dt.isocalendar()['week']

In [ ]:
temp = train[(train['건물번호'] == bno)].groupby(['month','day','weekday','week']).agg({'전력소비량(kWh)':['min','max']}).reset_index()



temp.columns = ['month','day','weekday','week', 'min','max']
temp['compare'] = temp['max'] - temp['min']
temp[temp.weekday == 2][-24:]
# temp[temp.month == 7][:]
temp[-24:]

In [ ]:
temp[temp.month==6]

In [ ]:
for i in range(0, 7):  
    plt.figure(figsize=(16,4))
    temp = test[(test['건물번호'] == bno) & (test['month'] == 8)][24*i: 24*(i+1)]
    display(temp[:1])
    plt.plot(temp['일시'], temp['풍속(m/s)'],linestyle='-', marker='o', label="풍속")
    plt.plot(temp['일시'], temp['강수량(mm)'],linestyle='-', marker='o', label="강수량")
    plt.plot(temp['일시'], temp['습도(%)_plot'],linestyle='-', marker='o', label="습도")
    plt.plot(temp['일시'], temp['기온(C)'],linestyle='-', marker='o', label="기온")
    plt.plot(temp['일시'], temp['전력소비량(kWh)_plot'],linestyle='-', label="전력소비량")
    
    plt.legend()
    plt.title(temp.weekday.unique())
    plt.show()

In [ ]:
for bno in range(37,44):    

    plt.figure(figsize=(16,4))

    temp = train[(train['건물번호'] == bno) & (train['month'] == 8)][-24:][['일시', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '전력소비량(kWh)_plot','weekday']].fillna(0) # 강수 0
    plt.plot(temp['일시'], temp['풍속(m/s)'],linestyle='-', marker='o', label="풍속")
    plt.plot(temp['일시'], temp['강수량(mm)'],linestyle='-', marker='o', label="강수량")
    plt.plot(temp['일시'], temp['습도(%)'],linestyle='-', marker='o', label="습도")
    plt.plot(temp['일시'], temp['전력소비량(kWh)_plot'],linestyle='-', label="전력소비량")
    
    plt.legend()
    title = str(bno) +'_'+ str(temp['weekday'].unique()[0])
    plt.title(title)
    plt.show()

    print()

In [ ]:
# for bno in building_no:
for bno in range(93,100):    

    plt.figure(figsize=(16,4))

    temp = train[(train['건물번호'] == bno) & (train['month'] == 8)][-48:][['일시', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '전력소비량(kWh)_plot']].fillna(0) # 강수 0
    plt.plot(temp['일시'], temp['풍속(m/s)'],linestyle='-', marker='o', label="풍속")
    plt.plot(temp['일시'], temp['강수량(mm)'],linestyle='-', marker='o', label="강수량")
    plt.plot(temp['일시'], temp['습도(%)'],linestyle='-', marker='o', label="습도")
    plt.plot(temp['일시'], temp['전력소비량(kWh)_plot'],linestyle='-', label="전력소비량")
    
    plt.legend()
    plt.title(bno)
    plt.show()

    temp = train[(train['건물번호'] == bno) & (train['month'] == 8)][-24:][['일시', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '전력소비량(kWh)_plot']].fillna(0) # 강수 0
    temp_test = test[(test['건물번호'] == bno) & (test['month'] == 8)][:24][['일시', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '전력소비량(kWh)_plot']].fillna(0) # 강수 0
    plt.figure(figsize=(16,4))

    plt.plot(temp['일시'], temp['풍속(m/s)'],linestyle='-', marker='o', label="풍속")
    plt.plot(temp['일시'], temp['강수량(mm)'],linestyle='-', marker='o', label="강수량")
    plt.plot(temp['일시'], temp['습도(%)'],linestyle='-', marker='o', label="습도")
    plt.plot(temp['일시'], temp['전력소비량(kWh)_plot'],linestyle='-', label="전력소비량")

    plt.plot(temp_test['일시'], temp_test['풍속(m/s)'],linestyle='-', marker='o', label="test")
    plt.plot(temp_test['일시'], temp_test['강수량(mm)'],linestyle='-', marker='o', label="test")
    plt.plot(temp_test['일시'], temp_test['습도(%)'],linestyle='-', marker='o', label="test")
    plt.plot(temp_test['일시'], temp_test['전력소비량(kWh)_plot'],linestyle='-', label="test")
        
    plt.legend()
    plt.show()
    print()
    print()

In [ ]:
train.sort_values(['전력소비량(kWh)'])[:2].T

In [ ]:
train[train['전력소비량(kWh)'] == 0]

In [ ]:
for i in range(20, 30):
    temp = train[(train['건물번호'] == 93) & (train['month'] == 6)][24*i:24*(i+1)][['일시', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '전력소비량(kWh)_plot']].fillna(0) # 강수 0
    plt.figure(figsize=(16,4))

    plt.plot(temp['일시'], temp['강수량(mm)'],linestyle='-', marker='o', label="강수량")
    plt.plot(temp['일시'], temp['습도(%)'],linestyle='-', marker='o', label="습도")
    plt.plot(temp['일시'], temp['전력소비량(kWh)_plot'],linestyle='-', label="전력소비량")
    plt.legend()

In [ ]:
for i in building_no:
    temp = train[train['건물번호'] == i][:24]
        
    plt.figure(figsize=(16,4))
    plt.subplot(1,2,1)
    plt.plot(temp['일시'], temp['전력소비량(kWh)_plot'],linestyle='-', marker='o', label="label")
    plt.plot(temp['일시'], temp['기온(C)'],linestyle='-', marker='o', label="temp")
    #plt.plot(temp['일시'], temp['기온_shift2'],linestyle='-', marker='o', label="shift")
    plt.title(temp[:1]['건물유형'].values[0])
    # plt.plot(temp['일시'], temp['습도(%)'],linestyle='-', marker='o', label="moi")
    
    plt.subplot(1,2,2)
    temp = train[train['건물번호'] == i][2016:2040]
    plt.plot(temp['일시'], temp['전력소비량(kWh)_plot'],linestyle='-', marker='o', label="label")
    plt.plot(temp['일시'], temp['기온(C)'],linestyle='-', marker='o', label="temp")
    
    plt.legend()
    plt.show()

In [ ]:
train.groupby(['건물유형'])['전력소비량(kWh)_plot'].max()

In [ ]:
temp = train[:48]


temp['기온_shift2'] = temp['기온(C)'].shift(-2)

plt.figure(figsize=(10,4))
# plt.plot(temp['일시'], temp['전력소비량(kWh)_plot'],linestyle='-', marker='o', label="label")
plt.plot(temp['일시'], temp['기온(C)'],linestyle='-', marker='o', label="temp")
plt.plot(temp['일시'], temp['기온_shift2'],linestyle='-', marker='o', label="shift")
# plt.plot(temp['일시'], temp['습도(%)'],linestyle='-', marker='o', label="moi")
plt.legend()

In [ ]:
temp = train[train['건물번호'] == 1]#[:24]
temp['yyyymmdd'] = temp['일시'].dt.strftime('%Y%m%d')
rain_day = pd.DataFrame(temp[temp['강수량(mm)'] > 0.1].yyyymmdd.unique(), columns=['yyyymmdd'])
rain_day['rain'] = 1

train['yyyymmdd'] = train['일시'].dt.strftime('%Y%m%d')
train = train.merge(rain_day, on='yyyymmdd', how='left').fillna(0)
train.drop('yyyymmdd', axis=1, inplace=True)

In [ ]:
col = '전력소비량(kWh)_plot'
temp = train[train['건물번호'] == 1]#[:24]
temp['yyyymmdd'] = temp['일시'].dt.strftime('%Y%m%d')
temp = temp.groupby('yyyymmdd').agg({col:['min','max']}).reset_index()
temp.columns = ['yyyymmdd', 'min', 'max']

plt.figure(figsize=(20,4))
plt.plot(temp['yyyymmdd'], temp['min'],linestyle='-', marker='o', label="min")
plt.plot(temp['yyyymmdd'], temp['max'],linestyle='-', marker='o', label="max")
plt.xticks(rotation=60)
plt.show()

In [ ]:
# ['20220606', '20220609', '20220610', '20220613', '20220615','20220616', '20220623', '20220624', '20220627', '20220628','20220629'
#, '20220630', '20220706', '20220708', '20220711','20220713', '20220714', '20220716', '20220720', '20220721','20220722', '20220723', '20220724', '20220730', '20220731',
# '20220801', '20220802', '20220803', '20220806', '20220807','20220808', '20220809', '20220810', '20220811', '20220813',
# '20220814', '20220815', '20220819', '20220820', '20220822','20220823']
temp[:10*2]

In [ ]:
temp = train[24:24*2]

plt.figure(figsize=(10,4))
plt.plot(temp['일시'], temp['전력소비량(kWh)_plot'],linestyle='-', marker='o', label="label")
plt.plot(temp['일시'], temp['기온(C)'],linestyle='-', marker='o', label="temp")
plt.plot(temp['일시'], temp['습도(%)'],linestyle='-', marker='o', label="moi")
plt.legend()

In [ ]:
train[train['건물번호'] == 2][:24*2].to_excel('test_2.xlsx')

In [ ]:
train['shift24'] = train['전력소비량(kWh)_plot'].shift(24)

train['shift24'] = train['전력소비량(kWh)_plot'].shift(24)
train[24:24*2]

In [ ]:
i=0
train[train['건물번호'] == 2][24*i:24*(i+1)]

## Train Data Pre-Processing

In [ ]:
# 빌딩 데이터 정리
## null 처리
building_info['태양광용량(kW)'] = building_info['태양광용량(kW)'].str.replace('-','0').astype(float)
building_info['ESS저장용량(kWh)'] = building_info['ESS저장용량(kWh)'].str.replace('-','0').astype(float)
building_info['PCS용량(kW)'] = building_info['PCS용량(kW)'].str.replace('-','0').astype(float)

In [ ]:
#시계열 특성을 학습에 반영하기 위해 일시를 월, 일, 시간으로 나눕니다
train['month'] = train['일시'].apply(lambda x : int(x[4:6]))
train['day'] = train['일시'].apply(lambda x : int(x[6:8]))
train['time'] = train['일시'].apply(lambda x : int(x[9:11]))

test['month'] = test['일시'].apply(lambda x : int(x[4:6]))
test['day'] = test['일시'].apply(lambda x : int(x[6:8]))
test['time'] = test['일시'].apply(lambda x : int(x[9:11]))

##### 2023-07-18-1
target_encoding = train.groupby(['건물번호','weekday','time']).agg({'전력소비량(kWh)':['min', 'mean','max']})
train = train.merge(target_encoding, on=['건물번호','weekday','time'], how='left')
test = test.merge(target_encoding, on=['건물번호','weekday','time'], how='left')

In [ ]:
#결측값을 0으로 채웁니다
train = train.fillna(0)
test = test.fillna(0)

In [ ]:
# 상관관계
#corr = train.corr()
#corr.to_excel('corr.xlsx')

In [ ]:
train_x = train.drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'])
train_y = train['전력소비량(kWh)']

test_x = test.drop(columns=['num_date_time', '일시'])

In [ ]:
# ont-hot
train_x = pd.get_dummies(train_x, columns=['건물유형','weekday'], drop_first=True)
test_x = pd.get_dummies(test_x, columns=['건물유형','weekday'], drop_first=True)

In [ ]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score
X_train, X_test, y_train, y_test = train_test_split(train_x , train_y ,test_size=0.2, shuffle=True, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

## Regression Model Fit

In [ ]:
%%time
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

In [ ]:
y_preds = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
mean_absolute_percentage_error(y_test, y_preds)

## Inference

In [ ]:
model = RandomForestRegressor(random_state=42)
model.fit(train_x, train_y)

In [ ]:
preds = model.predict(test_x)

## Submit

In [ ]:
submission['answer'] = preds
submission

In [ ]:
submission.to_csv('20230718-1.csv', index=False)